In [1]:
import torch

In [2]:
batch = 1
seqlen = 3
d_model = 32
d_head = 8
assert d_model % d_head == 0
heads = d_model // d_head

In [3]:
torch.manual_seed(42)
x = torch.randn(batch, seqlen, d_model)
W_Q = torch.randn(d_model, d_model)
W_K = torch.randn(d_model, d_model)
W_V = torch.randn(d_model, d_model)
W_O = torch.randn(d_model, d_model)
x.shape, x

(torch.Size([1, 3, 32]),
 tensor([[[ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784, -1.2345, -0.0431,
           -1.6047, -0.7521,  1.6487, -0.3925, -1.4036, -0.7279, -0.5594,
           -0.7688,  0.7624,  1.6423, -0.1596, -0.4974,  0.4396, -0.7581,
            1.0783,  0.8008,  1.6806,  1.2791,  1.2964,  0.6105,  1.3347,
           -0.2316,  0.0418, -0.2516,  0.8599],
          [-1.3847, -0.8712, -0.2234,  1.7174,  0.3189, -0.4245,  0.3057,
           -0.7746, -1.5576,  0.9956, -0.8798, -0.6011, -1.2742,  2.1228,
           -1.2347, -0.4879, -0.9138, -0.6581,  0.0780,  0.5258, -0.4880,
            1.1914, -0.8140, -0.7360, -1.4032,  0.0360, -0.0635,  0.6756,
           -0.0978,  1.8446, -1.1845,  1.3835],
          [ 1.4451,  0.8564,  2.2181,  0.5232,  0.3466, -0.1973, -1.0546,
            1.2780, -0.1722,  0.5238,  0.0566,  0.4263,  0.5750, -0.6417,
           -2.2064, -0.7508,  0.0109, -0.3387, -1.3407, -0.5854,  0.5362,
            0.5246,  1.1412,  0.0516,  0.7440, -0.4816, -1.0495, 

In [4]:
q = (x.__matmul__(W_Q)).view(batch, seqlen, heads, d_head).transpose(1, 2)
k = (x.__matmul__(W_K)).view(batch, seqlen, heads, d_head).transpose(1, 2)
v = (x.__matmul__(W_V)).view(batch, seqlen, heads, d_head).transpose(1, 2)

q.shape, k.shape, v.shape

(torch.Size([1, 4, 3, 8]), torch.Size([1, 4, 3, 8]), torch.Size([1, 4, 3, 8]))

In [5]:
attn_scores = q.__matmul__(torch.transpose(k, -2, -1))
attn_scores.shape

torch.Size([1, 4, 3, 3])

In [6]:
attn_weights = torch.softmax(attn_scores / d_head ** 0.5, dim=-1)
attn_weights.shape

torch.Size([1, 4, 3, 3])

In [7]:
attended_vals = attn_weights.__matmul__(v)
attended_vals.shape

torch.Size([1, 4, 3, 8])

In [8]:
output_heads_combined = attended_vals.transpose(1, 2).contiguous().view(batch, seqlen, d_model)
output_heads_combined.shape

torch.Size([1, 3, 32])

In [9]:
output_proj = output_heads_combined.__matmul__(W_O)
output_proj.shape

torch.Size([1, 3, 32])

---

In [10]:
W_QK = W_Q.__matmul__(torch.transpose(W_K, 0, 1))
W_QK.shape

torch.Size([32, 32])

In [11]:
W_OV = W_V.__matmul__(W_O)
W_OV.shape

torch.Size([32, 32])

In [12]:
a = torch.randn(3, 512)
a

tensor([[-0.1637, -1.2390, -1.0557,  ..., -0.3666, -1.3768, -0.6575],
        [ 1.8197, -0.5205,  1.4799,  ...,  0.0713, -0.8059, -0.3507],
        [ 1.1280,  0.0259, -0.1495,  ...,  0.7496, -1.2053,  1.4265]])

In [13]:
Q_ = a.__matmul__(torch.randn(512, 64))
Q_.shape

torch.Size([3, 64])

---

In [14]:
# some sample stuff
torch.manual_seed(42)
attn_scores = torch.randn(3, 3)
d_hidden = 8 # aka d_model
q_last_layer = 4


upper_triangular = torch.triu(torch.ones_like(attn_scores), 1)
mask_bool = upper_triangular.bool()[:, :]
attn_scores_w_causal_mask = torch.masked_fill(attn_scores, mask_bool, value=torch.tensor(float("-inf")))
attn_w = torch.softmax(attn_scores_w_causal_mask / q_last_layer ** 0.5, -1)
attn_scores_w_causal_mask, attn_scores, attn_w

(tensor([[ 0.3367,    -inf,    -inf],
         [ 0.2303, -1.1229,    -inf],
         [ 2.2082, -0.6380,  0.4617]]),
 tensor([[ 0.3367,  0.1288,  0.2345],
         [ 0.2303, -1.1229, -0.1863],
         [ 2.2082, -0.6380,  0.4617]]),
 tensor([[1.0000, 0.0000, 0.0000],
         [0.6630, 0.3370, 0.0000],
         [0.6029, 0.1453, 0.2518]]))

The following are assumed to be the attention scores, attention scores (with masking), and attention weights for the input "The cat sat".

```python
(tensor([[ 0.3367,    -inf,    -inf],
         [ 0.2303, -1.1229,    -inf],
         [ 2.2082, -0.6380,  0.4617]]),

 tensor([[ 0.3367,  0.1288,  0.2345],
         [ 0.2303, -1.1229, -0.1863],
         [ 2.2082, -0.6380,  0.4617]]),
         
 tensor([[1.0000, 0.0000, 0.0000],
         [0.6630, 0.3370, 0.0000],
         [0.6029, 0.1453, 0.2518]]))
```


Looking at the 3x3 attention weights matrix, I can picture this as:

|       | The   |  cat  | sat |
|-------|-------|-------|-------|
| The   | 1.0000 | 0.0000 | 0.0000 |
| cat   | 0.6630 | 0.3370 | 0.0000 |
| sat   | 0.6029 | 0.1453 | 0.2518 |


Here, I can think of it like this:

1. to understand "cat", how much will I have to look at the token "the"? --> exactly 1.000, so strong relevance for this computation.
2. to understand "sat", how much will I have to look at the token "the"? --> 0.6029, so a pretty solid relevance for this computation.
3. to understand "sat", how much will I have to look at the token "cat"? --> 0.1453, so an okay-ish relevance, not as strong as the one just before

### Now, attended values:

In [15]:
torch.manual_seed(42)
# assuming:
X_IN = torch.randn(3, d_hidden) # [seqlen, d_hidden]
W_V_ = torch.randn(d_hidden, d_hidden)
V = X_IN.__matmul__(W_V_)
W_O_ = torch.randn(d_hidden, d_hidden)

ATTENDED = attn_w.__matmul__(V)

print(f"""W_V: {W_V_.shape}
{W_V_}

V: {V.shape}
{V}

attended weights: {ATTENDED.shape}
{ATTENDED}
""")

W_V: torch.Size([8, 8])
tensor([[ 0.0349,  0.3211,  1.5736, -0.8455,  1.3123,  0.6872, -1.0892, -0.3553],
        [-1.4181,  0.8963,  0.0499,  2.2667,  1.1790, -0.4345, -1.3864, -1.2862],
        [-0.8371, -0.9224,  1.8113,  0.1606,  0.3672,  0.1754,  1.3852, -0.4459],
        [-1.2024,  0.7078, -1.0759,  0.5357,  1.1754,  0.5612, -0.4527, -0.7718],
        [ 0.1453,  0.2311,  0.0087, -0.1423,  0.1971, -1.1441,  0.3383,  1.6992],
        [ 2.8140,  0.3598, -0.0898,  0.4584, -0.5644,  1.0563, -1.4692,  1.4332],
        [ 0.7281, -0.7106, -0.6021,  0.9604,  0.4048, -1.3543, -0.4976,  0.4747],
        [-0.1976,  1.2683,  1.2243,  0.0981,  1.7423, -1.3527,  0.2191,  0.5526]])

V: torch.Size([3, 8])
tensor([[-3.3538, -2.6614,  5.1813, -0.1024,  0.1555, -0.1970, -0.2466, -2.8980],
        [-0.4393,  1.2727,  0.4840, -3.8536, -1.8261,  5.3907,  0.3668, -2.1857],
        [-0.3070, -0.7861,  2.5214, -1.5471, -1.1485,  1.0364,  2.6574, -0.5662]])

attended weights: torch.Size([3, 8])
tensor([[-3

### Now the output projection

In [16]:
OUTPUT_PROJECTION = ATTENDED.__matmul__(W_O_)
print(f"""{OUTPUT_PROJECTION.shape}
{OUTPUT_PROJECTION}""")

torch.Size([3, 8])
tensor([[ 10.8347,  -4.9027,   7.4389,  -2.5553,   9.3776,  -2.1711, -14.9518,
          -5.0850],
        [  8.7450,  -3.9095,   3.6260,  -2.4063,   5.4063,  -1.0348, -11.8651,
          -2.1552],
        [  9.2331,  -4.1688,   2.8769,  -2.0400,   4.9977,  -1.7839, -11.5206,
          -2.4551]])


In [17]:
residual_stream = X_IN + OUTPUT_PROJECTION
# this is bascially:
#   x_in + attended_val @ W_O                  (here, x_in = (enbeddings + pos_encodings), and attended_val @ W_O = output_projection)
# = x_in + (attention_weights @ V) @ W_O



# = x_in + ((Q @ K.T) @ (x_in @ W_V) @ W_O)    (simplifying the attn weights a little here, softmax and being divide by d_k ** 0.5 are omitted)
residual_stream.shape, residual_stream

(torch.Size([3, 8]),
 tensor([[ 12.7616,  -3.4154,   8.3396,  -4.6608,  10.0561,  -3.4056, -14.9948,
           -6.6897],
         [  9.1009,  -4.5961,   3.1327,  -2.1649,   4.2954,  -0.9433, -14.1820,
           -2.3720],
         [  8.9234,  -4.5645,   3.6803,  -2.6616,   4.4057,  -1.8470, -12.3491,
           -2.1242]]))

In [ ]:
# for, say, the word "sat" (3rd word in the sequence):
W_sat = residual_stream[2]

# this is:
#   residual["sat"] aka residual[2]
# = x_in["sat"] + attended_val["sat"] @ W_O
# 
# Let's assume our attention weight matrix was:
#      The     cat    sat
# The  1.00     0      0
# cat  0.74    0.26    0
# sat  0.20    0.50    0.30

# this means attention_weights["sat"] = [0,20, 0.50, 0.30]

# so, residual["sat"]:
# = x_in["sat"] + attended_val["sat"]                           @ W_O
# = x_in["sat"] + (attention_weights["sat"] @ V)                @ W_O
# = x_in["sat"] + ([0.2, 0.5, 0.3] @ [V_The, V_cat, V_sat])     @ W_O
# = x_in["sat"] + (0.2 * V_The + 0.5 * V_cat + 0.3 * V_sat)     @ W_O
# = x_in["sat"] + (0.2 * x_in["the"] + 0.5 * x_in["cat"] + 0.3 * x_in["sat"]) @ W_O


W_sat # x_in["sat"] + (0.2 * x_in["the"] + 0.5 * x_in["cat"] + 0.3 * x_in["sat"]) @ W_O

tensor([  8.9234,  -4.5645,   3.6803,  -2.6616,   4.4057,  -1.8470, -12.3491,
         -2.1242])

So basically, we can divide this process into 2 parts:

```python
# 1. The QK circuit:
Q @ K.T
= (x @ W_Q) @ (x @ W_K).T

# 2. The OV circuit:
(attention_weight @ V) @ W_O
= (attention_weight @ (x @ W_V) @ W_O)
= (attention_weight @ x) @ (W_V @ W_O)
```

Why talk about `W_OV = W_V @ W_O`? => For analysis and interpretability

Instead of thinking "first we transform by `W_V`, then later by `W_O`," we can think: "This attention head applies a single combined transformation `W_OV` to move information."